In [ ]:
# Check for running GPU
!nvidia-smi

In [ ]:
#run once for initial installs
! pip install farm-haystack
! pip install git+https://github.com/deepset-ai/haystack.git
! pip install urllib3==1.25.4
! pip install torch==1.6.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 122kB 5.6MB/s eta 0:00:01
     |████████████████████████████████| 215kB 8.5MB/s eta 0:00:01
     |████████████████████████████████| 327kB 9.5MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 143kB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 245kB 13.1MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 15.9MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 2.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.8MB/s  eta 0:00:01
     |████████████████████████████████| 225kB 12.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 6.9MB/s  eta 0:00:01
     |████████████████████████████████| 1.4MB 11.5MB/s eta 0:00:01
     |████████████████████████████████| 983kB 14.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 5.8MB/s  eta 0:00:01
     |████████████████████████████████| 92kB 6.9MB/s  eta 0:00:01
    

     |████████████████████████████████| 102kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 16.7MB/s eta 0:00:01
     |████████████████████████████████| 14.2MB 400kB/s  eta 0:00:01
     |████████████████████████████████| 81kB 6.3MB/s  eta 0:00:01
     |█████████████████████████████▊  | 721.7MB 5.7MB/s eta 0:00:106    |███▉                            | 92.6MB 22.8MB/s eta 0:00:30     |███████████▍                    | 275.4MB 7.7MB/s eta 0:01:06     |█████████████████████████████   | 705.5MB 5.9MB/s eta 0:00:12     |█████████████████████████████▋  | 718.7MB 4.4MB/s eta 0:00:14

In [4]:
#Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

In [10]:
# Connect to Elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

02/09/2021 00:19:43 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.381s]
02/09/2021 00:19:43 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:200 request:0.201s]


In [ ]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader

# Testing with tutorial data
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)
dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# Connecting point to preprocessing of as4
document_store.write_documents(dicts)

In [12]:
# Fast filter to narrow down text - Default BM25, can be cunstomised
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
# Reader to further scan with Hugging Face models
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)
reader = FARMReader(model_name_or_path="deepset/bert-large-uncased-whole-word-masking-squad2", use_gpu=True)

In [27]:
from haystack.pipeline import ExtractiveQAPipeline

# Original Finder deprecated, pipeline allows more flexibility
# prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_retriever=10, top_k_reader=5)

# top_k_retriever -> the more retriever the more document scanned in Reader, slower but higher hit rate
extractive_pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever) # Other options: Document Search, Generative, FAQ

In [34]:
question = input("What would you like to ask? ")
prediction = extractive_pipeline.run(query=question, top_k_retriever=10, top_k_reader=5)
# print_answers(prediction, details="all") #details: all, medium, minimal
# data format: {query,'answers':[{'answer','score','probability','context','document_id','offset','meta'}]}
print(prediction['answers'][0]['answer'])

What would you like to ask? the father of arya stark


02/09/2021 01:04:47 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.012s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  9.27 Batches/s]

Eddard
